In [2]:
import os

# Utilities
import Utils.folder_utils as folder_utils
import Utils.TIMIT_utils as TIMIT_utils
import Utils.image_utils as image_utils

# Sound utils
from Utils.nist_reader import NistReader
import Utils.sound_utils as sound_utils

# Spectrograms
from Utils.SpectrogramFactory import SpectrogramFactory
from Utils.Spectrogram import Spectrogram

# Image utils
from PIL import Image

%matplotlib inline

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
#sound
path_to_flac = "Data/TIMIT/timit/train"
path_to_flac_test_folder = "TestFolder/TestData/fcjf0"
path_to_flac_test_folder_track = "TestFolder/TestData/fcjf0/SA1.WAV"

#temp
temp_folder = "TestFolder/Temp"
temp_phonemes_folder = "TestFolder/TempPhonemes"
#temp_image_to_process = "/Volumes/Storage/processing.jpg"

In [4]:
pathes = folder_utils.reverse_folder(path_to_flac_test_folder, ".WAV")

In [5]:
pathes

['TestFolder/TestData/fcjf0/SA1.WAV',
 'TestFolder/TestData/fcjf0/SA2.WAV',
 'TestFolder/TestData/fcjf0/SI1027.WAV',
 'TestFolder/TestData/fcjf0/SI1657.WAV',
 'TestFolder/TestData/fcjf0/SI648.WAV',
 'TestFolder/TestData/fcjf0/SX127.WAV',
 'TestFolder/TestData/fcjf0/SX217.WAV',
 'TestFolder/TestData/fcjf0/SX307.WAV',
 'TestFolder/TestData/fcjf0/SX37.WAV',
 'TestFolder/TestData/fcjf0/SX397.WAV']

In [6]:
nistReader = NistReader()
spectrogramFactory = SpectrogramFactory()
phonemeOffset = 64 * 5

In [7]:
path = pathes[0]
#sentence
print path
phonemes = TIMIT_utils.parse_phoneme_file(path)
speaker = folder_utils.get_speaker_name(path)

temp_speaker_folder = os.path.join(temp_folder, speaker)
if not os.path.exists(temp_speaker_folder):
    os.makedirs(temp_speaker_folder)
    
#converting to wave
wav_file = nistReader.Nist2Wav(path, temp_speaker_folder)
#wav_file = SU.PCM2Wav(path, temp_speaker_folder)

for i in range(len(phonemes)):
    phoneme = phonemes[i]
    #Cutting one phoneme
    if i == 0 or i == len(phonemes):
        start = int(phoneme[0])
        end = int(phoneme[1])
    else:
        start = int(phoneme[0]) - phonemeOffset
        end = int(phoneme[1]) + phonemeOffset
        
    # TODO: create a phoneme object
    phone_file = sound_utils.cutPhonemeChunk(wav_file, temp_phonemes_folder, start, end, phoneme[2])
    spectrogram = spectrogramFactory.create_spectrogram(phone_file)

TestFolder/TestData/fcjf0/SA1.WAV


In [8]:
########
# TEST #
########

phoneme = phonemes[len(phonemes) / 2]

start = int(phoneme[0]) - phonemeOffset
end = int(phoneme[1]) + phonemeOffset
        
phone_file = sound_utils.cutPhonemeChunk(wav_file, temp_phonemes_folder, start, end, phoneme[2])

In [9]:
spectrogramFactory = SpectrogramFactory()
spectrogram = spectrogramFactory.create_spectrogram(phone_file)

In [10]:
spectrogram.spectrogram_values.shape

(512, 12)

In [11]:
import numpy as np
posFreqs = spectrogram.spectrogram_values[0:256+1, :];
negFreqs = np.flipud(spectrogram.spectrogram_values[256:,:])

powerSpec = posFreqs;
freqRange = np.arange(1,256+1);
powerSpec[freqRange,:] = np.multiply(powerSpec[freqRange,:], negFreqs)

magSpec = np.sqrt(powerSpec);

In [12]:
powerSpec[0,:]

array([ 0.00836100+0.j,  0.00727717+0.j,  0.01003497+0.j,  0.00887806+0.j,
       -0.00262595+0.j, -0.00737807+0.j, -0.00295518+0.j,  0.00227111+0.j,
        0.00260382+0.j,  0.00302338+0.j,  0.00058657+0.j, -0.00329179+0.j])

In [13]:
powerSpec[2,:]

array([  1.40251771e-07 +1.10970685e-07j,
         4.33206354e-08 -6.28852186e-08j,
         6.36172587e-08 +7.60844158e-08j,
         8.45347214e-07 -3.05491849e-07j,
        -5.67254972e-07 -1.19910090e-06j,
        -4.80158619e-07 +3.01476181e-07j,
         1.17051773e-09 +2.71565076e-09j,
         1.04544812e-08 +7.70091429e-08j,
         2.01785987e-07 +7.55665629e-08j,
         7.86670623e-09 -3.70170552e-08j,
        -1.03176258e-08 -4.87147601e-08j,   3.02313104e-10 -3.45361207e-10j])

In [ ]:
self._fftSize = 2*self.window_size
fftB = int(math.floor(self.window_size/2))
fftE = int(fftB + self.window_size)
fftBuffer = np.zeros(self._fftSize)

self._frameCount = int(math.floor((self._wave_form.size - self.window_size)/self.frame_step) + 1)
spectrum = np.zeros((self._fftSize, self._frameCount));
spectrum = spectrum + 0j

h = 0.54 - 0.46*np.cos(2*math.pi* np.arange(self.window_size)/(self.window_size-1))
for frameNumber in range(0,self._frameCount):
    waveB = (frameNumber)*self.frame_step
    waveE = waveB + self.window_size
    fftBuffer = 0*fftBuffer #Make sure the entire buffer is empty
    fftBuffer[fftB:fftE] = np.multiply(self._wave_form[waveB:waveE], h)
    fftBuffer = np.fft.fftshift(fftBuffer)
    spectrum[:,frameNumber] = np.fft.fft(fftBuffer)